# Experiments for LICS paper

In [1]:
%%bash
ltl3ba -v
ltl3hoa -v
ltl2tgba --version
ltlcross --version

LTL3BA 1.1.3
LTL3HOA 1.0.1
ltl2tgba (spot 2.2.2.dev)

Copyright (C) 2017  Laboratoire de Recherche et Développement de l'Epita.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
ltlcross (spot 2.2.2.dev)

Copyright (C) 2017  Laboratoire de Recherche et Développement de l'Epita.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


In [2]:
from ltlcross_runner import LtlcrossRunner
from experiments_lib import has_f_merging, has_g_merging
from IPython.display import display
import pandas as pd
import spot
import sys
spot.setup(show_default='.a')

In [3]:
def generate(n=100,func=(lambda x: True),filename=None):
    if filename is None:
        file_h = sys.stdout
    else:
        file_h = open(filename,'w')
    f = spot.randltl(['a','b','c','d'],
                     ltl_priorities='M=0,W=0,xor=0',
                     simplify=3,tree_size=15).relabel_bse(spot.Abc)\
                     .unabbreviate('WM')
    i = 0
    printed = set()
    while(i < n):
        form = next(f)
        #form = spot.negative_normal_form(form)
        if form in printed:
            continue
        if func(form) and not form.is_tt() and not form.is_ff():
            print(form,file=file_h)
            printed.add(form)
            i += 1

## Impact of F- and G-merging

In [4]:
rerun = False

### Generating Formulae

We generate random formulae that are both F- and G- mergable.  We use the strict control for merging (a formula is generated only if some states will be merged) so we can emphasise the effect of the modified construction. 

In [5]:
MI_f = 'formulae/merging_impact.ltl'
generate(1000,lambda x : has_g_merging(x) and has_f_merging(x),MI_f)

### Alternating automata

In [6]:
### Tools' setting ###
ltl3hoa_shared = "ltl3hoa -p1 -t0 -u0 -z0 -f %f -a3 "
end = " > %O"
tools = {"LTL3HOA_both_merging"    : ltl3hoa_shared + end,
         "LTL3HOA_F-merging"  : ltl3hoa_shared + "-G0 " + end,
         "LTL3HOA_G-merging"  : ltl3hoa_shared + "-F0" + end,
         "LTL3HOA_no_merging" : ltl3hoa_shared + "-F0 -G0" + end,
         #"LTL3BA-POST"     : "ltl3ba -C -P -A -H1 -f %s > %O",
        }
MI_order = ["LTL3HOA_no_merging","LTL3HOA_F-merging",
            "LTL3HOA_G-merging","LTL3HOA_both_merging"]
### File with measured statistics ###
merging_imp_alt = 'MI_alt.csv'

In [7]:
rerun = False

In [8]:
MI_alt = LtlcrossRunner(tools,res_filename=merging_imp_alt,
                        formula_files=[MI_f],
                        cols=["states","transitions"])
if rerun:
    MI_alt.run_ltlcross()
MI_alt.parse_results()

Here are the cumulative number of states and number of nondeterministic automata (without branching transitions) for each tool. 

In [9]:
t1_alt = MI_alt.values.stack(level=0).unstack().sum().unstack().loc[MI_order,:]
t1_alt

,states,transitions
tool,,
LTL3HOA_no_merging,7610.0,59859.0
LTL3HOA_F-merging,6469.0,49663.0
LTL3HOA_G-merging,6284.0,50007.0
LTL3HOA_both_merging,5108.0,40068.0


### Nondeterministic automata

In [10]:
### Tools' setting ###
ltl3hoa_shared = "ltl3hoa -p2 -s0 -u0 -z0 -f %f -a3 "
end = " > %O"
tools = {"LTL3HOA_both_merging"    : ltl3hoa_shared + end,
         "LTL3HOA_F-merging"  : ltl3hoa_shared + "-G0 " + end,
         "LTL3HOA_G-merging"  : ltl3hoa_shared + "-F0" + end,
         "LTL3HOA_no_merging" : ltl3hoa_shared + "-F0 -G0" + end,
         #"LTL3BA-POST"     : "ltl3ba -S0 -H2 -R -P -A -f %s > %O",
        }
### File with measured statistics ###
merging_imp = 'MI_nondet.csv'

In [11]:
MI_nondet = LtlcrossRunner(tools,res_filename=merging_imp,
                           formula_files=[MI_f],
                           cols=["states","transitions","nondet_aut"])
if rerun:
    MI_nondet.run_ltlcross()
MI_nondet.parse_results()

Here are the cumulative number of states and number of nondeterministic automata (without branching transitions) for each tool. 

In [12]:
t1_nondet = MI_nondet.values.stack(level=0).unstack().sum().unstack().loc[MI_order,:]
t1_nondet

,states,transitions,nondet_aut
tool,,,
LTL3HOA_no_merging,7822.0,64172.0,988.0
LTL3HOA_F-merging,6686.0,55610.0,821.0
LTL3HOA_G-merging,7870.0,64436.0,987.0
LTL3HOA_both_merging,6031.0,49814.0,824.0


In [13]:
t1 = pd.concat([t1_alt,t1_nondet],axis=1,keys=['Alternating','Nondeterministic'])
t1

Alternating             Nondeterministic              \
                          states transitions           states transitions   
tool                                                                        
LTL3HOA_no_merging        7610.0     59859.0           7822.0     64172.0   
LTL3HOA_F-merging         6469.0     49663.0           6686.0     55610.0   
LTL3HOA_G-merging         6284.0     50007.0           7870.0     64436.0   
LTL3HOA_both_merging      5108.0     40068.0           6031.0     49814.0   

                                 
                     nondet_aut  
tool                             
LTL3HOA_no_merging        988.0  
LTL3HOA_F-merging         821.0  
LTL3HOA_G-merging         987.0  
LTL3HOA_both_merging      824.0

In [14]:
row_map={"LTL3HOA_no_merging" : 'no merging',
         "LTL3HOA_F-merging"  : '$\F$-merging',
         "LTL3HOA_G-merging"  : '$\G$-merging',
         "LTL3HOA_both_merging" : 'full-merging'}
#t1.index.names = out_rows
t1.rename(row_map,inplace=True)

In [15]:
print(t1.to_latex(float_format='{:.0f}'.format,escape=False)
      ,file=open('lics_t1.tex','w'))

## Comparison with LTL to TGBA translators

In [16]:
rerun = False

### Generating Formulae

We generate random formulae where there is some $\mathsf{F}\psi$ or $\mathsf{G}\psi$ such that $\psi$ contains some other temporal formula.

In [17]:
comp_f = 'formulae/comparison.ltl'
generate(1000,lambda x:
        has_f_merging(x,False) or has_g_merging(x,False),
        comp_f)

### Nondeterministic automata

In [18]:
### Tools' setting ###
ltl3hoa_shared = "ltl3hoa -p2 -f %f -a3 "
end = " > %O"
tools = {"LTL3HOA_both_merging" : ltl3hoa_shared + end,
         "LTL3BA"               : 'ltl3ba -H2 -f %s | autfilt --small > %O',         
         "SPOT-det"             : 'ltl2tgba --deterministic -H %f>%O',
         "SPOT-small"           : 'ltl2tgba --small -H %f>%O',
        }
tgba = ["LTL3BA","SPOT-det","SPOT-small"]
### File with measured statistics ###
comp_res = 'comp_nondet.csv'

In [19]:
comp = LtlcrossRunner(tools,res_filename=comp_res,
                           formula_files=[comp_f],
                           cols=["states","transitions","nondet_aut"])
if rerun:
    comp.run_ltlcross()
comp.parse_results()

Here are the cumulative number of states and number of nondeterministic automata (without branching transitions) for each tool. 

In [20]:
comp.compute_best(tgba,'VB(TGBA)')
comp.compute_best(None,'VB(All)')

In [21]:
tmp = comp.values.stack(level=0).unstack().sum()
t2 = tmp.unstack().loc[tgba+['LTL3HOA_both_merging']+comp.mins,:]
t2

,states,transitions,nondet_aut
tool,,,
LTL3BA,4645.0,29215.0,760.0
SPOT-det,4560.0,29082.0,705.0
SPOT-small,4531.0,28948.0,721.0
LTL3HOA_both_merging,4580.0,31779.0,743.0
VB(TGBA),4525.0,28484.0,705.0
VB(All),4172.0,26858.0,617.0


In [22]:
print(t2.to_latex(float_format='{:.0f}'.format,bold_rows=True,
                 ))#,file=open('lics_t2.tex','w'))

\begin{tabular}{lrrr}
\toprule
{} &  states &  transitions &  nondet\_aut \\
tool                 &         &              &             \\
\midrule
LTL3BA               &    4645 &        29215 &         760 \\
SPOT-det             &    4560 &        29082 &         705 \\
SPOT-small           &    4531 &        28948 &         721 \\
LTL3HOA\_both\_merging &    4580 &        31779 &         743 \\
VB(TGBA)             &    4525 &        28484 &         705 \\
VB(All)              &    4172 &        26858 &         617 \\
\bottomrule
\end{tabular}

